# Table 2: Ocean Heat Content linear trends

This notebook will reproduce table 2 from *Ocean Heat Content responses to changing Anthropogenic Aerosol Forcing Strength: regional and multi-decadal variability*, E. Boland et al. 2022 (doi to come). This will require utils.py (expects to find it in ../code) and input datafiles (expects to find them in ../data_in) to run - please see the README for details. Output is saved to a tex file named 'table2.tex', automatically in ../figs/

The data files loaded were created as follows:
- ohc\_\[exp\]\_\[run\]\_\[basin\].nc and ohc\_bydepth\_\[exp\]\_\[run\]\_\[basin\].nc from ohc_by_basin_depth.py
- ohc_pic_all_drift.nc from ohc_by_basin_depth_pic.py followed by ohc_pic_drift.py

Please attribute any plots or code from this notebook using the DOI from Zenodo: TO COME

E Boland Feb 2022 [emmomp@bas.ac.uk](email:emmomp@bas.ac.uk)

In [1]:
import xarray as xr
import sys
sys.path.insert(0,'../code/')
import utils

In [2]:
# Directories for saving plots and finding input data
tables_dir = '../figs/' # Where you want the table saved
data_dir='../data_in/' # Where the input data is (see README)

# Experiment info, don't alter
exps=['historical0p2','historical0p4','historical0p7','historical1p0','historical1p5']
runs=['r1i1p1f1','r2i1p1f1','r3i1p1f1','r4i1p1f1','r5i1p1f1']
basins=['global','atl','pac','so','ind']

In [5]:
ohc_basin_fd=[]
for basin in basins:
    ohc_temp=[]
    for exp in exps:
        ohc_temp.append(xr.open_mfdataset(data_dir+'ohc_tseries/ohc_'+exp+'*'+basin+'*.nc',concat_dim='run',combine='nested',data_vars=['ohc',], coords='minimal', compat='override'))
    ohc_temp=xr.combine_nested(ohc_temp,concat_dim='exp',coords='minimal',data_vars=['ohc',]) 
    ohc_basin_fd.append(ohc_temp)
ohc_basin_fd=xr.concat(ohc_basin_fd,'basin',coords='minimal',data_vars=['ohc',],compat='override') 

In [8]:
ohc_basin=[]
for basin in basins:
    ohc_global=[]
    for exp in exps:
        ohc_global.append(xr.open_mfdataset(data_dir+'ohc_tseries/ohc_bydepth_'+exp+'*'+basin+'*.nc',concat_dim='run',combine='nested',data_vars=['ohc',]))
    ohc_global=xr.combine_nested(ohc_global,concat_dim='exp',data_vars=['ohc',]) 
    ohc_basin.append(ohc_global)
ohc_basin=xr.concat(ohc_basin,'basin',data_vars=['ohc',],compat='override',coords='minimal') 

In [9]:
ohc_basin_fd['deptht_bins']=(('deptht_bins'),[b'Full Depth',])
ohc_all = xr.concat([ohc_basin_fd,ohc_basin],'deptht_bins')
ohc_all=ohc_all.swap_dims({'time_counter':'time_centered'})

In [10]:
ohc_delta=(ohc_all.sel(time_centered=slice('1995-01-01',None)).mean(dim='time_centered'))-(ohc_all.sel(time_centered=slice(None,'1870-01-01')).mean(dim='time_centered'))
ohc_delta=ohc_delta/1e22/((2005-1860)/100)
ohc_delta['forc']=('exp',[0.38,0.6,0.93,1.17,1.5])
ohc_delta.ohc.attrs['long_name']='OHC'
ohc_delta.ohc.attrs['units']='x10$^{22}$ J/century'   
ohc_delta.forc.attrs['long_name']='-(Aerosol Equivalent Radiative Forcing)'
ohc_delta.forc.attrs['units']='W/m$^2$'   

In [11]:
# Stack run and experiment dimensions to get all 25 ensemble members 
XX=ohc_delta.forc.broadcast_like(ohc_delta.ohc).stack({'run_exp':['run','exp']}).chunk({'run_exp': -1})
YY=ohc_delta.ohc.stack({'run_exp':['run','exp']}).chunk({'run_exp': -1})
stats_out=utils.lin_regress(XX,YY,[['run_exp'],['run_exp']]) #Calls a vectorised version of scipy.stats.linregress

In [12]:
r2=stats_out.sel(parameter='r_value')**2
r2['parameter']='r^2'
stats_out=xr.concat([stats_out,r2],'parameter')

In [13]:
stats_out=stats_out.compute()

In [19]:
#Create latex formatted table and write to file
nd=stats_out.deptht_bins.size

with open(tables_dir+'table2.tex','w') as f:
    print('&&\\textbf{Global} & \\textbf{Atlantic} & \\textbf{Pacific} & \\textbf{Southern} & \\textbf{Indian}  \\\\',file=f)
    for id in range(0,nd):
        depth_label=stats_out['deptht_bins'].values[id]
        r2=stats_out.sel(parameter='r^2').isel(deptht_bins=id).data
        slopes=stats_out.sel(parameter='slope').isel(deptht_bins=id).data
        errs=stats_out.sel(parameter='std_err').isel(deptht_bins=id).data
        str='\\textbf{{{}}} & $R^2$ & \\textbf{{{:0.2f}}}  &  \\textbf{{{:0.2f}}}   &  \\textbf{{{:0.2f}}}   &   \\textbf{{{:0.2f}}}    &  \\textbf{{{:0.2f}}}  \\\\\
    & Slope  & ${:0.1f}\\pm{:0.1f}$& ${:0.1f}\\pm{:0.1f}$& ${:0.1f}\\pm{:0.1f}$ & ${:0.1f}\\pm{:0.1f}$ & ${:0.1f}\pm{:0.1f}$  \\\\'.format(depth_label,r2[0],r2[1],r2[2],r2[3],r2[4],slopes[0],errs[0],slopes[1],errs[1],slopes[2],errs[2],slopes[3],errs[3],slopes[4],errs[4])
        if id ==1:
            print('\\hline \\hline',file=f)
        else:
            print('\\hline',file=f)
        print(str,file=f)
    